# benchmarking using osx & radeon gpu on keras + plaidml

https://medium.com/@danbrice.datascience/deep-learning-on-a-mac-with-amd-gpu-4be1f18944a

### prep

In [1]:
import keras
import os
import numpy as np

/Users/jj/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train.shape

(60000, 28, 28)

In [3]:
test = 'hello'
y_train.shape

(60000,)

In [4]:
test

'hello'

In [5]:
X_test.shape

(10000, 28, 28)

In [6]:
# one hot
n_categories = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, n_categories)
y_test = keras.utils.to_categorical(y_test, n_categories)

y_train.shape

(60000, 10)

In [7]:
# normalize
def normalize(X):
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    X = X.astype('float32')
    X /= 255
    
    return X

X_train = normalize(X_train)
X_test = normalize(X_test)

X_test.shape

(10000, 28, 28, 1)

In [8]:
def get_mnist_cnn_model(n_categories):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters=32, 
                                  kernel_size=(3,3), 
                                  activation='relu',
                                  input_shape=(28,28,1)))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Conv2D(filters=64,
                                 kernel_size=(3,3),
                                 activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Conv2D(filters=64,
                                 kernel_size=(3,3),
                                 activation='relu'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64,
                                activation='relu'))
    model.add(keras.layers.Dense(n_categories,
                                activation='softmax'))


    model.compile(loss=keras.losses.categorical_crossentropy,
                 optimizer=keras.optimizers.Adam(),
                 metrics=['accuracy'])
    
    return model

### params

In [14]:
EPOCHS = 15
BATCH_SIZE = 100

## CPU w/ tensorflow

In [10]:
keras.backend.backend()

'tensorflow'

In [11]:
keras.backend.tensorflow_backend._get_available_gpus()

[]

In [12]:
model_cpu = get_mnist_cnn_model(n_categories)

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
%%time

model_cpu.fit(X_train, y_train,
             batch_size=BATCH_SIZE,
             epochs=EPOCHS,
             verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
60000/60000 [==============================] - 26s 439us/step - loss: 0.2194 - acc: 0.9354
Epoch 2/15
60000/60000 [==============================] - 25s 420us/step - loss: 0.0519 - acc: 0.9836
Epoch 3/15
60000/60000 [==============================] - 25s 415us/step - loss: 0.0376 - acc: 0.9885
Epoch 4/15
60000/60000 [==============================] - 25s 419us/step - loss: 0.0293 - acc: 0.9907
Epoch 5/15
60000/60000 [==============================] - 25s 417us/step - loss: 0.0234 - acc: 0.9927
Epoch 6/15
60000/60000 [==============================] - 25s 414us/step - loss: 0.0196 - acc: 0.9938
Epoch 7/15
60000/60000 [==============================] - 25s 408us/step - loss: 0.0164 - acc: 0.9949
Epoch 8/15
60000/60000 [==============================] - 25s 414us/step - loss: 0.0133 - acc: 0.9958
Epoch 9/15
60000/60000 [==============================] - 25s 409us/step - loss: 0.0124 - acc: 0.9958
Epoch 10/15
60000/60000 [=========

In [14]:
model_cpu.evaluate(X_test, y_test, verbose=1)

10000/10000 [==============================] - 1s 111us/step


[0.0314761482036105, 0.9913]

## GPU w/ plaidml

In [39]:
os.environ['KERAS_BACKEND'] = 'plaidml.keras.backend'

import importlib
importlib.reload(keras.backend)

Using plaidml.keras.backend backend.


<module 'keras.backend' from '/Users/jj/anaconda3/lib/python3.6/site-packages/keras/backend/__init__.py'>

In [15]:
keras.backend.backend()

'plaidml.keras.backend'

In [16]:
model_gpu = get_mnist_cnn_model(n_categories)

In [17]:
%%time

model_gpu.fit(X_train, y_train,
             batch_size=BATCH_SIZE,
             epochs=EPOCHS,
             verbose=1)

Epoch 1/15
60000/60000 [==============================] - 11s 185us/step - loss: 0.2101 - acc: 0.9372
Epoch 2/15
60000/60000 [==============================] - 10s 169us/step - loss: 0.0570 - acc: 0.9823
Epoch 3/15
60000/60000 [==============================] - 10s 172us/step - loss: 0.0398 - acc: 0.9876
Epoch 4/15
60000/60000 [==============================] - 10s 170us/step - loss: 0.0303 - acc: 0.9905
Epoch 5/15
60000/60000 [==============================] - 10s 169us/step - loss: 0.0256 - acc: 0.9916
Epoch 6/15
60000/60000 [==============================] - 10s 169us/step - loss: 0.0214 - acc: 0.9930
Epoch 7/15
60000/60000 [==============================] - 10s 169us/step - loss: 0.0184 - acc: 0.9939
Epoch 8/15
60000/60000 [==============================] - 10s 169us/step - loss: 0.0149 - acc: 0.9951
Epoch 9/15
60000/60000 [==============================] - 10s 171us/step - loss: 0.0138 - acc: 0.9953
Epoch 10/15
60000/60000 [==============================] - 10s 169us/step - loss: 

In [18]:
model_gpu.evaluate(X_test, y_test, verbose=1)

10000/10000 [==============================] - 1s 140us/step


[0.03283998546600342, 0.9918]

# conclusion

Wall time comps:
- cpu via tensorflow backend: 6min 11s
- gpu via plaidml backend: 2min 33s